In [1]:
import google.cloud.aiplatform as aip
import kfp

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'assessment1-361706'

In [3]:
BUCKET_URI = "gs://vertex-ai-1"
SERVICE_ACCOUNT = "vertex-ai@assessment1-361706.iam.gserviceaccount.com"
PIPELINE_ROOT = "{}/pipeline_root/price-regular".format(BUCKET_URI)

In [4]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [5]:
TRAIN_FILE_NAME = "promo.csv"
! gsutil cp gs://vertex-ai-1/promo.csv {PIPELINE_ROOT}/data/

In [6]:
gcs_csv_path = f"{PIPELINE_ROOT}/data/{TRAIN_FILE_NAME}"
REGION = "us-central1"

Copying gs://vertex-ai-1/promo.csv [Content-Type=text/csv]...
/ [0 files][    0.0 B/ 20.8 KiB]                                                
/ [0 files][ 20.8 KiB/ 20.8 KiB]                                                
-
- [1 files][ 20.8 KiB/ 20.8 KiB]                                                

Operation completed over 1 objects/20.8 KiB.                                     


In [7]:
import random
import string

def generate_uuid():
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=8))

UUID = generate_uuid()

In [11]:
@kfp.dsl.pipeline(name="automl-tab-training-v2")
def pipeline(project: str = PROJECT_ID, region: str = REGION):
    from google_cloud_pipeline_components import aiplatform as gcc_aip
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)

    dataset_create_op = gcc_aip.TabularDatasetCreateOp(
        project=project, display_name="price-promo", gcs_source=gcs_csv_path
    )

    training_op = gcc_aip.AutoMLTabularTrainingJobRunOp(
        project=project,
        display_name="train-automl-price-promo",
        optimization_prediction_type="regression",
        optimization_objective="minimize-rmse",
        budget_milli_node_hours = 1000,
        column_transformations=[
            {"numeric": {"column_name": "cost_price"}},
            {"numeric": {"column_name": "promo_price"}},
            {"numeric": {"column_name": "promo_volume"}},
            {"numeric": {"column_name": "promo_pe"}}
        ],
        dataset=dataset_create_op.outputs["dataset"],
        target_column="promo_pe",
    )

    endpoint_op = EndpointCreateOp(
        project=project,
        location=region,
        display_name="train-automl-pprice_endpoint",
    )

    _ = ModelDeployOp(
        model=training_op.outputs["model"],
        endpoint=endpoint_op.outputs["endpoint"],
        dedicated_resources_machine_type="n1-standard-4",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

In [12]:
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="tabular_regression_ppipeline.json",
)

In [13]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "assessment1-361706-0a442522753f.json"

In [14]:
DISPLAY_NAME = "price-promo_" + UUID

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tabular_regression_ppipeline.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

job.run()

! rm tabular_regression_ppipeline.json

Creating PipelineJob
PipelineJob created. Resource name: projects/1050316944664/locations/us-central1/pipelineJobs/automl-tab-training-v2-20220906194425
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1050316944664/locations/us-central1/pipelineJobs/automl-tab-training-v2-20220906194425')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/automl-tab-training-v2-20220906194425?project=1050316944664
PipelineJob projects/1050316944664/locations/us-central1/pipelineJobs/automl-tab-training-v2-20220906194425 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1050316944664/locations/us-central1/pipelineJobs/automl-tab-training-v2-20220906194425 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1050316944664/locations/us-central1/pipelineJobs/automl-tab-training-v2-20220906194425 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/10

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
from google.cloud import aiplatform

aiplatform.get_pipeline_df(pipeline = 'automl-tab-training-v2')

,pipeline_name,run_name,param.input:region,param.input:project
0,automl-tab-training-v2,automl-tab-training-v2-20220906194425,us-central1,assessment1-361706
1,automl-tab-training-v2,automl-tab-training-v2-20220906184004,us-central1,assessment1-361706
2,automl-tab-training-v2,automl-tab-training-v2-20220906183142,us-central1,assessment1-361706
3,automl-tab-training-v2,automl-tab-training-v2-20220906180341,us-central1,assessment1-361706
4,automl-tab-training-v2,automl-tab-training-v2-20220906174631,us-central1,assessment1-361706


In [16]:
models = aiplatform.Model.list()

In [17]:
model = models[0]
model.resource_name

'projects/1050316944664/locations/us-central1/models/7275261732807770112'

In [18]:
endpoint = aiplatform.Endpoint.list()[0]
endpoint.display_name

'train-automl-pprice_endpoint'

In [20]:
prediction = endpoint.predict(instances=[{"cost_price": "180", "promo_price": "300", "promo_volume": "290"}])
prediction.predictions[0]['value']

-0.1657071262598038

# GET Predictions with REST

In [21]:
import json
targets = [{"cost_price": "180", "promo_price": "300", "promo_volume": "290"}]
with open(f'request.json','w') as file:
    file.write(json.dumps({"instances": targets}))

In [22]:
ENDPOINT_ID="2318641324561006592"
PROJECT_ID="assessment1-361706"
INPUT_DATA_FILE="request.json"
!gcloud auth application-default login

^C


In [ ]:
curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-central1/endpoints/${ENDPOINT_ID}:predict \
-d "@${INPUT_DATA_FILE}"